# Exercise 2: Introducing TiGL Geometry Functionality

## Goal: Access and manipulate a CPACS-based geometry using the TiGL geometry kernel with OCC classes

### Steps (some of them are already prepared):

**Load the configuration:**

- Open CPACS with TiXI3. 
- Open geometry with TiGL3. 
- Get the configuration from the `CCPACSConfigurationManager`. 

- Access geometric components (wing/fuselage/fuel tank) via configuration or UIDMgr.
- Get the wing component by UID.


**Prepare 3D visualization (OCC)**

**Execute geometric operations:**

- Create a primitive box (OCC)
- Transform the box (TiGL)

- Wrap the box as CNamedShape for boolean operations (TiGL) 

- Boolean cut: subtract box from wing (TiGL)


**Display results**

In [1]:
#do the imports
from tixi3 import tixi3wrapper
from tigl3 import tigl3wrapper

import tigl3.configuration

from tigl3.boolean_ops import CCutShape
from tigl3.geometry import CTiglTransformation
from tigl3.geometry import CNamedShape
from tigl3.geometry import get_face

from OCC.Core.BRepPrimAPI import BRepPrimAPI_MakeBox
from OCC.Core.gp import gp_Dir

from OCC.Display.WebGl.jupyter_renderer import JupyterRenderer
from OCC.Extend.DataExchange import read_step_file, export_shape_to_svg

import os

## Open the CPACS file with TIXI

In [2]:
#specify data directory and filename
data_dir = os.path.join("..", "data")
file = "simpletest.cpacs.xml"

In [3]:
#create handles
tixi_h = tixi3wrapper.Tixi3()
tigl_h = tigl3wrapper.Tigl3()

#initialize handles
tixi_h.open(os.path.join(data_dir, file))
tigl_h.open(tixi_h, "")

## Load the Configuration and CCAPCSConfigurationManager

In [4]:
#load the configuration manager
mgr = tigl3.configuration.CCPACSConfigurationManager_get_instance()

#load the configuration
config = mgr.get_configuration(tigl_h._handle.value)

#get uid manager
uid_mgr = config.get_uidmanager()

## Create a primitive box with OCC

In [5]:
'''
create a Box named 'box' with BRepPrimAPI_MakeBox(x,y,z).Solid()
'''
tmp_box = BRepPrimAPI_MakeBox(0.2, 0.8, 0.5)

'''
Retrieve geometric component from configuration
'''
tmp_wing = uid_mgr.get_geometric_component("Wing")

# transform the objects to be usable for TiGL and the display function
box = tmp_box.Solid()
wing = tmp_wing.get_loft()

## Create a JupyterRenderer object to view the created shapes in this exercise

In [6]:
'''
open display
'''
display_box = JupyterRenderer()
display_wing = JupyterRenderer()

#help(display)

In [7]:
# display the plain wing
display_wing.DisplayShape(
    wing.shape(),
    render_edges=True,
    edge_color="white",
    edge_deflection=0.1,
    transparency=True,
    opacity = 0.5,
    topo_level="Face",
    shape_color="green",
    update=True,
    selectable=True)

In [8]:
# display the plain box
display_box.DisplayShape(
    box,
    render_edges=True,
    edge_color="white",
    edge_deflection=0.1,
    transparency=True,
    opacity = 0.5,
    topo_level="Face",
    shape_color="pink",
    update=True,
    selectable=True)

## Use TiGL to translate the box

In [9]:
#uncomment to show TiGL Options
#help(CTiglTransformation)

In [10]:
'''
move Box
'''
#uncomment to show TiGL Options
#help(CTiglTransformation)

#create transformation
trafo = CTiglTransformation()
trafo.add_translation(0.8, 0.6, -0.15)

#apply translation to the box, creating a new translated object
#help(trafo)
moved_box = trafo.transform(box)

In [11]:
#display moved box

display_moved_box = JupyterRenderer()

display_moved_box.DisplayShape(
    moved_box,
    render_edges=True,
    edge_color="white",
    edge_deflection=0.1,
    transparency=True,
    opacity = 0.5,
    topo_level="Face",
    shape_color="yellow",
    update=True,
    selectable=True)

## Use TiGL to make the box a cutout shape 

In [12]:
'''
make box a cutout shape
'''
# Create CNamedShape
named_box = CNamedShape(moved_box, "CutOut")

'''
apply cutout shape to the wing by creating a cut shape
'''
#cut box from wing
cutter = CCutShape(wing, named_box)
cut_wing_shape = cutter.named_shape()

## Display the cutout shape

In [13]:
'''
Now check by looking at our file via OCC Display, if the booloean operation worked:
'''

display_cutout = JupyterRenderer()

# Color the new created faces and egdes differently
for i in range(0, cut_wing_shape.get_face_count()):
    ft = cut_wing_shape.get_face_traits(i)
    if ft.origin().name() == "CutOut":
        color = "cyan"
    if ft.origin().name() == "Wing":
        color = "green"
    display_cutout.DisplayShape(get_face(cut_wing_shape.shape(), i), shape_color=color, render_edges=False, edge_color="blue", edge_deflection=0.05, vertex_color=None, quality=2.0, transparency=True, opacity=0.5, topo_level='default', update=False, selectable=True)

display_cutout.Display()